# Fairness in Classification

- 📺 **Video:** [https://youtu.be/N4f2-S19LME](https://youtu.be/N4f2-S19LME)

## Overview
- Quantify how traditional accuracy metrics can obscure disparities across demographic groups.
- Explore mitigation levers such as reweighting decisions after training a classifier.

## Key ideas
- **Group fairness metrics:** compare positive prediction rates, true positive rates, and false positive rates between groups.
- **Bias amplification:** data imbalance or label bias can cause models to widen gaps seen in the training set.
- **Post-processing:** calibration or threshold adjustment can partially correct disparities without retraining.
- **Trade-offs:** improving one fairness criterion can harm others, so context-sensitive choices are necessary.

## Demo
Train logistic regression on a synthetic dataset with a sensitive attribute, then measure accuracy, demographic parity, and equal opportunity gaps as discussed in the lecture (https://youtu.be/N4f2-S19LME).

In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

rng = np.random.default_rng(12)
num_samples = 600
group = rng.integers(0, 2, size=num_samples)
feature1 = rng.normal(loc=group * 0.8, scale=1.0, size=num_samples)
feature2 = rng.normal(size=num_samples)
linear = 0.6 * feature1 + 0.3 * feature2 - 0.5 * group
probs = 1 / (1 + np.exp(-linear))
y = (probs > 0.5).astype(int)

X = np.column_stack([feature1, feature2, group])
clf = LogisticRegression(max_iter=1000, solver='lbfgs')
clf.fit(X, y)

preds = clf.predict(X)
print(classification_report(y, preds, digits=3))

def group_stats(name, mask):
    g_truth = y[mask]
    g_pred = preds[mask]
    positive_rate = g_pred.mean()
    true_positive = ((g_pred == 1) & (g_truth == 1)).sum()
    positives = (g_truth == 1).sum()
    tpr = true_positive / positives if positives else 0.0
    false_positive = ((g_pred == 1) & (g_truth == 0)).sum()
    negatives = (g_truth == 0).sum()
    fpr = false_positive / negatives if negatives else 0.0
    print(f"{name}: positive_rate={positive_rate:.3f} | TPR={tpr:.3f} | FPR={fpr:.3f}")

print()
print('Group metrics:')
for g in [0, 1]:
    group_stats(f"Group {g}", group == g)
print()
print('Demographic parity gap:', abs(preds[group==0].mean() - preds[group==1].mean()))


              precision    recall  f1-score   support

           0      0.977     0.977     0.977       307
           1      0.976     0.976     0.976       293

    accuracy                          0.977       600
   macro avg      0.977     0.977     0.977       600
weighted avg      0.977     0.977     0.977       600


Group metrics:
Group 0: positive_rate=0.470 | TPR=0.950 | FPR=0.000
Group 1: positive_rate=0.505 | TPR=1.000 | FPR=0.043

Demographic parity gap: 0.03476719688778407


## Try it
- Modify the demo
- Add a tiny dataset or counter-example


## References
- [Eisenstein 4.2](https://github.com/jacobeisenstein/gt-nlp-class/blob/master/notes/eisenstein-nlp-notes.pdf)
- [Multiclass lecture note](https://www.cs.utexas.edu/~gdurrett/courses/online-course/multiclass.pdf)
- [A large annotated corpus for learning natural language inference](https://www.aclweb.org/anthology/D15-1075/)
- [Authorship Attribution of Micro-Messages](https://www.aclweb.org/anthology/D13-1193/)
- [50 Years of Test (Un)fairness: Lessons for Machine Learning](https://arxiv.org/pdf/1811.10104.pdf)
- [[Article] Amazon scraps secret AI recruiting tool that showed bias against women](https://www.reuters.com/article/us-amazon-com-jobs-automation-insight/amazon-scraps-secret-ai-recruiting-tool-that-showed-bias-against-women-idUSKCN1MK08G)
- [[Blog] Neural Networks, Manifolds, and Topology](http://colah.github.io/posts/2014-03-NN-Manifolds-Topology/)
- [Eisenstein Chapter 3.1-3.3](https://github.com/jacobeisenstein/gt-nlp-class/blob/master/notes/eisenstein-nlp-notes.pdf)
- [Dropout: a simple way to prevent neural networks from overfitting](https://dl.acm.org/doi/10.5555/2627435.2670313)
- [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/abs/1502.03167)
- [Adam: A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980)
- [The Marginal Value of Adaptive Gradient Methods in Machine Learning](https://papers.nips.cc/paper/2017/hash/81b3833e2504647f9d794f7d7b9bf341-Abstract.html)


*Links only; we do not redistribute slides or papers.*